<a href="https://colab.research.google.com/github/LejoFlores/CdA_WatershedAnalysis/blob/main/Process_CdA_Subwatersheds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Installs geemap package
import subprocess

try:
    import geemap
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

# Checks whether this notebook is running on Google Colab
try:
    import google.colab
    import geemap.eefolium as emap
except:
    import geemap as emap

# Authenticates and initializes Earth Engine
import ee

try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()  

geemap package not installed. Installing ...
To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=XmzMKWlFuIg52Vyb4WC9992pjvKXvOcUlzufxvXuKsE&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AY0e-g7Nq0vwHKY2VVSa96RGsVyvwbBZ4SLdUhyMHdMhcja7xDafLMtgm4s

Successfully saved authorization token.


In [ ]:
# Get the USGS HUC08 collection from the data catalog
huc08 = ee.FeatureCollection('USGS/WBD/2017/HUC08')

# Get the three basins within the Northwest that correspond to the Lake Coeur d'Alene Watershed
# Find the correspondence between the basin name and codes here: https://water.usgs.gov/GIS/huc.html

# 4-Digit HUCs:
# 17010301 -- Upper Coeur d'Alene. Idaho. Area = 905 sq.mi.
# 17010302 -- South Fork Coeur d'Alene. Idaho. Area = 297 sq.mi.
# 17010303 -- Coeur d'Alene Lake. Idaho, Washington. Area = 663 sq.mi.
# 17010304 -- St. Joe. Idaho. Area = 1860 sq.mi.

up_cda_watershed = huc08.filter(ee.Filter.inList('huc8', ee.List(['17010301'])))
sf_cda_watershed = huc08.filter(ee.Filter.inList('huc8', ee.List(['17010302'])))
lk_cda_watershed = huc08.filter(ee.Filter.inList('huc8', ee.List(['17010303'])))
st_joe_watershed = huc08.filter(ee.Filter.inList('huc8', ee.List(['17010304'])))

# Get the whole watershed
cda_watershed = huc08.filter(ee.Filter.inList('huc8', ee.List(['17010301','17010302','17010303','17010304'])))

# Now union the three basins to get one feature that consists of the entire Snake River
cda_watershed_full = cda_watershed.union()

# Create a new map and add the watershed we will use to clip the EVI data
Map = emap.Map(center=(47.5, -116.25),zoom=9)

Map.addLayer(ee.Image().paint(up_cda_watershed, 0, 2), {}, 'Upper Coeur d\'Alene River')
Map.addLayer(ee.Image().paint(sf_cda_watershed, 0, 2), {}, 'South Fork Coeur d\'Alene River')
Map.addLayer(ee.Image().paint(lk_cda_watershed, 0, 2), {}, 'Lake Coeur d\'Alene River')
Map.addLayer(ee.Image().paint(st_joe_watershed, 0, 2), {}, 'St. Joe River')

Map.addLayerControl()
Map

In [ ]:
task = ee.batch.Export.table.toDrive(**{
    'collection': up_cda_watershed,
    'description': 'upper_cda_watershed',
    'driveFolder': '_CdA_GEE_data', 
    'fileFormat': 'KML',
})
task.start()
task.status()

task = ee.batch.Export.table.toDrive(**{
    'collection': sf_cda_watershed,
    'description': 'sfork_cda_watershed',
    'driveFolder': '_CdA_GEE_data', 
    'fileFormat': 'KML',
})
task.start()
task.status()

task = ee.batch.Export.table.toDrive(**{
    'collection': lk_cda_watershed,
    'description': 'lake_cda_watershed',
    'driveFolder': '_CdA_GEE_data', 
    'fileFormat': 'KML',
})
task.start()
task.status()

task = ee.batch.Export.table.toDrive(**{
    'collection': st_joe_watershed,
    'description': 'stjoe_watershed',
    'driveFolder': '_CdA_GEE_data', 
    'fileFormat': 'KML',
})
task.start()
task.status()

task = ee.batch.Export.table.toDrive(**{
    'collection': cda_watershed_full,
    'description': 'cda_watershed',
    'driveFolder': '_CdA_GEE_data', 
    'fileFormat': 'KML',
})
task.start()
task.status()


{'creation_timestamp_ms': 1620540104122,
 'description': 'cda_watershed',
 'id': 'TXSDZRQB6CZFLAI5E3IYIKHC',
 'name': 'projects/earthengine-legacy/operations/TXSDZRQB6CZFLAI5E3IYIKHC',
 'start_timestamp_ms': 0,
 'state': 'READY',
 'task_type': 'EXPORT_FEATURES',
 'update_timestamp_ms': 1620540104122}

In [ ]:
CdA_NED = ee.Image('USGS/NED').select('elevation').clip(cda_watershed_full)

up_cda_ned = ee.Image('USGS/NED').select('elevation').clip(up_cda_watershed)
sf_cda_ned = ee.Image('USGS/NED').select('elevation').clip(sf_cda_watershed)
lk_cda_ned = ee.Image('USGS/NED').select('elevation').clip(lk_cda_watershed)
st_joe_ned = ee.Image('USGS/NED').select('elevation').clip(st_joe_watershed)

NED_Vis = {
  'min': 0.0,
  'max': 4000.0,
  'gamma': 1.6,
}

Map2 = emap.Map(center=(47.5, -116.25),zoom=9)

Map2.addLayer(CdA_NED, NED_Vis, 'Elevation')
Map2

In [ ]:
downConfig = {'scale': 10, 'maxPixels': 1.0E13, 'driveFolder': '_CdA_GEE_data', 'crs':'EPSG:4326'}  # scale means resolution.

task_up_cda = ee.batch.Export.image(up_cda_ned.clip(up_cda_watershed.geometry()), 'up_cda_ned', downConfig)
task_up_cda.start()
task_up_cda.status()

task_sf_cda = ee.batch.Export.image(sf_cda_ned.clip(sf_cda_watershed.geometry()), 'sf_cda_ned', downConfig)
task_sf_cda.start()
task_sf_cda.status()

task_lk_cda = ee.batch.Export.image(lk_cda_ned.clip(lk_cda_watershed.geometry()), 'lk_cda_ned', downConfig)
task_lk_cda.start()
task_lk_cda.status()

task_st_joe = ee.batch.Export.image(st_joe_ned.clip(st_joe_watershed.geometry()), 'st_joe_ned', downConfig)
task_st_joe.start()
task_st_joe.status()

task_cda = ee.batch.Export.image(CdA_NED.clip(cda_watershed_full.geometry()), 'cda_ned', downConfig)
task_cda.start()
task_cda.status()


{'creation_timestamp_ms': 1620540131444,
 'description': 'cda_ned',
 'id': 'JMPNDCESNYCWYOJGOAISIQMZ',
 'name': 'projects/earthengine-legacy/operations/JMPNDCESNYCWYOJGOAISIQMZ',
 'start_timestamp_ms': 0,
 'state': 'READY',
 'task_type': 'EXPORT_IMAGE',
 'update_timestamp_ms': 1620540131444}

In [ ]:
dataset = ee.ImageCollection('USGS/NLCD').filterBounds(cda_watershed_full)
cda_landcover = dataset.select('landcover') \
                    .map(lambda image: image.clip(cda_watershed_full.geometry()))

landcoverVis = {
  'min': 0.0,
  'max': 95.0,
  'palette': ['000000','000000','000000','000000','000000','000000','000000',
    '000000','000000','000000','000000','466b9f','d1def8','000000','000000',
    '000000','000000','000000','000000','000000','000000','dec5c5','d99282',
    'eb0000','ab0000','000000','000000','000000','000000','000000','000000',
    'b3ac9f','000000','000000','000000','000000','000000','000000','000000',
    '000000','000000','68ab5f','1c5f2c','b5c58f','000000','000000','000000',
    '000000','000000','000000','000000','af963c','ccb879','000000','000000',
    '000000','000000','000000','000000','000000','000000','000000','000000',
    '000000','000000','000000','000000','000000','000000','000000','000000',
    'dfdfc2','d1d182','a3cc51','82ba9e','000000','000000','000000','000000',
    '000000','000000','dcd939','ab6c28','000000','000000','000000','000000',
    '000000','000000','000000','b8d9eb','000000','000000','000000','000000',
    '6c9fb8']}

Map = emap.Map(center=(47.5, -116.25),zoom=9)
Map.addLayer(cda_landcover, landcoverVis, 'Landcover')
Map

In [ ]:
downConfig = {'scale': 30, 'maxPixels': 1.0E13, 'driveFolder': '_CdA_GEE_data', 'crs':'EPSG:4326'}  # scale means resolution.
img_lst = cda_landcover.toList(100)

count = cda_landcover.size().getInfo()

for i in range(0, count):
    image = ee.Image(img_lst.get(i)).unmask(-9999)
    name = 'CDA_'+image.get('system:index').getInfo()
    print(name)
    task = ee.batch.Export.image(image.clip(cda_watershed_full.geometry()), name, downConfig)
    task.start()
    print(task.status())

CDA_NLCD1992
{'state': 'READY', 'description': 'CDA_NLCD1992', 'creation_timestamp_ms': 1620540161149, 'update_timestamp_ms': 1620540161149, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': '2YFX2OQ6RXCXSJADPNYYQGKY', 'name': 'projects/earthengine-legacy/operations/2YFX2OQ6RXCXSJADPNYYQGKY'}
CDA_NLCD2001
{'state': 'READY', 'description': 'CDA_NLCD2001', 'creation_timestamp_ms': 1620540168077, 'update_timestamp_ms': 1620540168077, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'H3OMUDTTDUWH6ZB2GSPPIPBQ', 'name': 'projects/earthengine-legacy/operations/H3OMUDTTDUWH6ZB2GSPPIPBQ'}
CDA_NLCD2004
{'state': 'READY', 'description': 'CDA_NLCD2004', 'creation_timestamp_ms': 1620540170660, 'update_timestamp_ms': 1620540170660, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'O2NUTPE2NMHPNWMPC2L64QIB', 'name': 'projects/earthengine-legacy/operations/O2NUTPE2NMHPNWMPC2L64QIB'}
CDA_NLCD2006
{'state': 'READY', 'description': 'CDA_NLCD2006', 'creation_timestamp_ms': 

In [ ]:
ee.data.getTaskStatus('JMPNDCESNYCWYOJGOAISIQMZ')

[{'attempt': 1,
  'creation_timestamp_ms': 1620540131444,
  'description': 'cda_ned',
  'id': 'JMPNDCESNYCWYOJGOAISIQMZ',
  'name': 'projects/earthengine-legacy/operations/JMPNDCESNYCWYOJGOAISIQMZ',
  'start_timestamp_ms': 1620540181928,
  'state': 'RUNNING',
  'task_type': 'EXPORT_IMAGE',
  'update_timestamp_ms': 1620540243858}]